In [8]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                          11/2020     Adrien Poindron    #
###########################################################

# Imports

In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [3]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [4]:
from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.signal import savgol_filter

In [43]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
# %run ../../Functions/1.2/data_fct_Adrien_point_by_point
%run ../GiantMol/Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 25})

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.00e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          b# écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [81]:
renorm = 0.5*m_GM/C_e

# Sélection fichiers et chargement des données

In [8]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

In [10]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

('J:/20200903/All_stat_DC01_RF08_50eV.npz', 'J:/20200903/All_stat_DC01_RF09_50eV.npz', 'J:/20200903/All_stat_DC01_RF10_50eV.npz', 'J:/20200903/All_stat_DC01_RF11_50eV.npz', 'J:/20200903/All_stat_DC03_RF08_50eV.npz', 'J:/20200903/All_stat_DC03_RF09_50eV.npz', 'J:/20200903/All_stat_DC03_RF10_50eV.npz', 'J:/20200903/All_stat_DC03_RF11_50eV.npz', 'J:/20200903/All_stat_DC05_RF08_50eV.npz', 'J:/20200903/All_stat_DC05_RF09_50eV.npz', 'J:/20200903/All_stat_DC05_RF10_50eV.npz', 'J:/20200903/All_stat_DC05_RF11_50eV.npz', 'J:/20200903/All_stat_DC07_RF08_50eV.npz', 'J:/20200903/All_stat_DC07_RF09_50eV.npz', 'J:/20200903/All_stat_DC07_RF10_50eV.npz', 'J:/20200903/All_stat_DC07_RF11_50eV.npz', 'J:/20200903/All_stat_DC09_RF08_50eV.npz', 'J:/20200903/All_stat_DC09_RF09_50eV.npz', 'J:/20200903/All_stat_DC09_RF10_50eV.npz', 'J:/20200903/All_stat_DC09_RF11_50eV.npz', 'J:/20200903/All_stat_DC11_RF08_50eV.npz', 'J:/20200903/All_stat_DC11_RF09_50eV.npz', 'J:/20200903/All_stat_DC11_RF10_50eV.npz', 'J:/202009

In [44]:
# load check
time         = []
Temperature  = []
Temp_var     = []
Fluorescence = []
Fluo_var     = []
SNR          = []
Ray          = []
Len          = []
DeltaEc      = []
condition    = []

for outfile in file_path:
    with load(outfile) as data:
        time.append(data['time'])
        Temperature.append(data['Temperature'])
        Temp_var.append(data['Temp_var'])
        Fluorescence.append(data['Fluo'])
        Fluo_var.append(data['Fluo_var'])
        SNR.append(data['SNR'])
        Ray.append(data['Ray'])
        Len.append(data['Len'])
        DeltaEc.append(data['DeltaEc'])
        condition.append(data['condition'])

DeltaEc = multiply(DeltaEc,renorm)
# Temperature,Temp_var,
# Fluo, Fluo_var, SNR,
# Ray, Len,
# DeltaEc,
# condition

In [129]:
detect_rate = []

for k,l in enumerate(SNR):
    alors = 0
    for m,n in enumerate(l):
        if n>=1:
            alors += 1
    detect_rate.append(alors)

In [63]:
print(condition)
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

[array('DC01_RF08', dtype='<U9'), array('DC01_RF09', dtype='<U9'), array('DC01_RF10', dtype='<U9'), array('DC01_RF11', dtype='<U9'), array('DC03_RF08', dtype='<U9'), array('DC03_RF09', dtype='<U9'), array('DC03_RF10', dtype='<U9'), array('DC03_RF11', dtype='<U9'), array('DC05_RF08', dtype='<U9'), array('DC05_RF09', dtype='<U9'), array('DC05_RF10', dtype='<U9'), array('DC05_RF11', dtype='<U9'), array('DC07_RF08', dtype='<U9'), array('DC07_RF09', dtype='<U9'), array('DC07_RF10', dtype='<U9'), array('DC07_RF11', dtype='<U9'), array('DC09_RF08', dtype='<U9'), array('DC09_RF09', dtype='<U9'), array('DC09_RF10', dtype='<U9'), array('DC09_RF11', dtype='<U9'), array('DC11_RF08', dtype='<U9'), array('DC11_RF09', dtype='<U9'), array('DC11_RF10', dtype='<U9'), array('DC11_RF11', dtype='<U9'), array('DC18_RF08', dtype='<U9'), array('DC18_RF09', dtype='<U9'), array('DC18_RF10', dtype='<U9'), array('DC18_RF11', dtype='<U9')]


# Traitement des données

In [12]:
shape(time),shape(Temperature),shape(Fluorescence)

((28, 100, 20850), (28, 100, 20850, 3), (28, 100, 19860))

In [24]:
cond_num = len(condition)

plot_point = 18
plot_try   = 2

tt    = time[plot_point][plot_try]
T_all = Temperature[plot_point][plot_try]
PM    = Fluorescence[plot_point][plot_try]

print(shape(tt),shape(T_all),shape(PM))

n_dt = 100
dt  = 2*pi / (n_dt*Omega)
i_free__fly_Lan = 1000*n_dt
i_laser_fly = 6000*n_dt
i_laser_fly_quench = 8000*n_dt
print(dt)
diff_time = len(tt)-len(PM)
print(diff_time)

(20850,) (20850, 3) (19860,)
5e-09
990


In [25]:
# Plot Delta E en fct L
# Avec discri detec et non detec

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Plot temperature Langevin + md laser  ALL SimuType  ALL TEMP MEAN
matplotlib.rcParams.update({'font.size': 25})

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[7, 3]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,savgol_filter(mean(T_all,axis=1),25,1), label='Tx',color='red',lw=2.5)

time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax1.vlines((ta[2])*1e3,1e-4,1e4,color='black')
# ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

ax1.annotate('1', xy=(0.22,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('2', xy=(2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('3', xy=((3.5+7.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
# ax1.annotate('4', xy=((7.5+7.57)/2,2.5e-4), xycoords='data',
# size=15, ha='center', va='top', color='xkcd:black',
# bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('5', xy=((7.5+10.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)

# ax2.plot(tt_md*1e3,PM, label='Tx')

tt_1ms_step = tt[diff_time::2000]
PM_1ms = PM[::2000]
tt_1us_step = tt[diff_time::2]
PM_1us = PM[::2]

ax2.scatter(tt_1ms_step*1e3,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
ax2.legend()
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,150,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,150,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('PM signal')

left, bottom, width, height = [0.345, 0.565, 0.35, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,mean(T_all,axis=1), label='Tx',color='red',lw=2.5,marker='x')
# axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
# axins.vlines(ta[2]*1e3,1e-4,1e4,color='black')

# axins.add_patch(
#      patches.Rectangle(
#         (t_in*1000, 3e-4),
#         (t_out - t_in)*1000,
#         0.1,
#         edgecolor ='k',
#         facecolor =  cc('grey',0.5),
#         fill=True
#      ) )

x1, x2, y1, y2 = 7.44, 7.63, 0.00012, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# axins.yaxis.tick_right()

locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.setp(axins.get_xticklabels(), backgroundcolor="w")
plt.setp(axins.get_yticklabels(), backgroundcolor="w")

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")

axins.annotate('3', xy=(7.48,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('4', xy=((7.5+7.53)/2,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('5', xy=(7.55,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.00005, 1000)
ax2.set_ylim(-0, 135)

plt.tight_layout()
subplots_adjust(hspace=0.0,bottom=0.11,left=0.12,right=0.98,top=0.97)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:139: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [112]:
# Len.append(data['Len'])
# DeltaEc.append(data['DeltaEc'])

figname = 'DE_vs_L'
fig, ax = plt.subplots(1, 1,num=figname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(14,6)

for i,j in enumerate(condition):
    ii = my_order[i]
    print(ii,j)
    for k,l in enumerate(Len[ii][:]):
        if SNR[ii][k] > 1:
            ax.scatter(l*1e3,DeltaEc[ii][k]*1000,color='xkcd:green',marker='3')
        else:
            ax.scatter(l*1e3,DeltaEc[ii][k]*1000,color='xkcd:rouge',marker='4')
    ax.text(mean(Len[ii])*1e3,max(DeltaEc[ii][:])*1000+8-1.5**(i%4), ii,
            fontsize='10',ha='center')
    ax.vlines(mean(Len[ii])*1e3,max(DeltaEc[ii][:])*1000+8-1.5**(i%4),max(DeltaEc[ii][:])*1000+0.5,ls=':')
# ax.set_xlabel(r'$\langle L \rangle /2$'
ax.set_xlabel(r'$ L /2$ [µm]')
ax.set_ylabel(r'$\Delta E_c$ [meV]')
ax.set_ylim(-76,9)
ax.grid()

tight_layout()

savefig(figname+'.png',dpi=600)

0 DC01_RF08
1 DC01_RF09
2 DC01_RF10
3 DC01_RF11
4 DC03_RF08
5 DC03_RF09
6 DC03_RF10
7 DC03_RF11
8 DC05_RF08
9 DC05_RF09
10 DC05_RF10
11 DC05_RF11
12 DC07_RF08
13 DC07_RF09
14 DC07_RF10
15 DC07_RF11
16 DC09_RF08
17 DC09_RF09
18 DC09_RF10
19 DC09_RF11
24 DC11_RF08
25 DC11_RF09
26 DC11_RF10
27 DC11_RF11
20 DC18_RF08
21 DC18_RF09
22 DC18_RF10
23 DC18_RF11


In [157]:
# Plot Delta en fct omega_r

points_and_coord = {'DC01_RF08': ['01', '08'],
 'DC01_RF09': ['01', '09'],
 'DC01_RF10': ['01', '10'],
 'DC01_RF11': ['01', '11'],
 'DC03_RF08': ['03', '08'],
 'DC03_RF09': ['03', '09'],
 'DC03_RF10': ['03', '10'],
 'DC03_RF11': ['03', '11'],
 'DC05_RF08': ['05', '08'],
 'DC05_RF09': ['05', '09'],
 'DC05_RF10': ['05', '10'],
 'DC05_RF11': ['05', '11'],
 'DC07_RF08': ['07', '08'],
 'DC07_RF09': ['07', '09'],
 'DC07_RF10': ['07', '10'],
 'DC07_RF11': ['07', '11'],
 'DC09_RF08': ['09', '08'],
 'DC09_RF09': ['09', '09'],
 'DC09_RF10': ['09', '10'],
 'DC09_RF11': ['09', '11'],
 'DC11_RF08': ['11', '08'],
 'DC11_RF09': ['11', '09'],
 'DC11_RF10': ['11', '10'],
 'DC11_RF11': ['11', '11'],
 'DC18_RF08': ['18', '08'],
 'DC18_RF09': ['18', '09'],
 'DC18_RF10': ['18', '10'],
 'DC18_RF11': ['18', '11']}

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

x = []
y = []
z = []

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])
    
# calcul des paramètres Mathieu et frequences propres

wr = np.zeros((len(points_and_coord)))
wz = np.zeros((len(points_and_coord)))
w_ratio = np.zeros((len(points_and_coord)))

for i,j in enumerate(points_and_coord):
    # voltage recovery
    coucou_DC = int(j[j.find('DC')+2:j.find('DC')+4])
    coucou_RF = int(j[j.find('RF')+2:j.find('RF')+4])
#     print(Udc[coucou_DC],'   ',Urf[coucou_RF])
    ax = 4*(C_e/m_Ca)*Udc[coucou_DC]/(r0**2*Omega**2)
    az = 8*mkappa*(C_e/m_Ca)*Udc[coucou_DC]/(d0**2*Omega**2)
    qx = -4*(C_e/m_Ca)*Urf[coucou_RF]/(r0**2*Omega**2)
    wr[i] = Omega/2*sqrt(qx**2/2+ax-0.5*az)/(2*pi)
    wz[i] = sqrt(2*mkappa*C_e*Udc[coucou_DC]/(m_Ca*d0**2))/(2*pi)
    w_ratio[i] = wz[i]/wr[i]
print(i,wr)

# Len.append(data['Len'])
# DeltaEc.append(data['DeltaEc'])

figname = 'DE_vs_wr'
fig, ax = plt.subplots(1, 1,num=figname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(14,6)

for i,j in enumerate(condition):
    ii = my_order[i]
    print(ii,j)
    for k,l in enumerate(DeltaEc[ii]):
        if SNR[ii][k] > 1:
            ax.scatter(wr[i]*1e-3,l*1000,color='xkcd:green',marker='3')
        else:
            ax.scatter(wr[i]*1e-3,l*1000,color='xkcd:rouge',marker='4')
    ax.text(wr[i]*1e-3+1,max(DeltaEc[ii][:])*1000+1, i,
            fontsize='10',ha='center')
#     ax.vlines(wr[i]*1e-3+3,max(DeltaEc[ii][:])*1000+2.8,max(DeltaEc[ii][:])*1000+0.2,ls=':')
# ax.set_xlabel(r'$\langle L \rangle /2$'
ax.set_xlabel(r'$f_r$ [kHz]')
ax.set_ylabel(r'$\Delta E_c$ [meV]')
ax.set_ylim(-76,9)
ax.grid()

tight_layout()

savefig(figname+'.png',dpi=600)

27 [379801.19433665 416173.71922269 452666.58862231 489320.9523725
 387941.77547966 423615.90918474 459518.13282799 495666.08555912
 395915.00995173 430929.59106702 466269.00855675 501931.01350312
 403730.81260643 438121.20058262 472923.52688702 508118.7029219
 411398.15628004 445196.65356191 479485.69970963 514231.94202792
 426319.35315694 459020.48780883 492347.72682685 526245.41621504
 418925.19252652 452161.40291437 485959.2680141  520273.35521298]
0 DC01_RF08
1 DC01_RF09
2 DC01_RF10
3 DC01_RF11
4 DC03_RF08
5 DC03_RF09
6 DC03_RF10
7 DC03_RF11
8 DC05_RF08
9 DC05_RF09
10 DC05_RF10
11 DC05_RF11
12 DC07_RF08
13 DC07_RF09
14 DC07_RF10
15 DC07_RF11
16 DC09_RF08
17 DC09_RF09
18 DC09_RF10
19 DC09_RF11
24 DC11_RF08
25 DC11_RF09
26 DC11_RF10
27 DC11_RF11
20 DC18_RF08
21 DC18_RF09
22 DC18_RF10
23 DC18_RF11


In [154]:
# Plot f_r en fct L/2

figname = 'wr_vs_L'
fig, ax = plt.subplots(1, 1,num=figname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(14,6)

for i,j in enumerate(condition):
    ii = my_order[i]
    print(ii,j)
    for k,l in enumerate(Len[ii][:]):
        if SNR[ii][k] > 1:
            ax.scatter(l*1e3,wr[i]*1e-3,color='xkcd:green',marker='3')
        else:
            ax.scatter(l*1e3,wr[i]*1e-3,color='xkcd:rouge',marker='4')
    ax.text(l*1e3,wr[i]*1e-3+4, ii,
            fontsize='10',ha='center')
#     ax.vlines(wr[i]*1e-3+3,max(DeltaEc[ii][:])*1000+2.8,max(DeltaEc[ii][:])*1000+0.2,ls=':')
# ax.set_xlabel(r'$\langle L \rangle /2$'
ax.set_xlabel(r'$ L/2$ [µm]')
ax.set_ylabel(r'$f_r$ [kHz]')
ax.set_ylim(365,545)
ax.grid()

tight_layout()

savefig(figname+'.png',dpi=600)

0 DC01_RF08
1 DC01_RF09
2 DC01_RF10
3 DC01_RF11
4 DC03_RF08
5 DC03_RF09
6 DC03_RF10
7 DC03_RF11
8 DC05_RF08
9 DC05_RF09
10 DC05_RF10
11 DC05_RF11
12 DC07_RF08
13 DC07_RF09
14 DC07_RF10
15 DC07_RF11
16 DC09_RF08
17 DC09_RF09
18 DC09_RF10
19 DC09_RF11
24 DC11_RF08
25 DC11_RF09
26 DC11_RF10
27 DC11_RF11
20 DC18_RF08
21 DC18_RF09
22 DC18_RF10
23 DC18_RF11


In [146]:
shape(Len)

(28, 100)